In [9]:
# %%writefile infer_fuse.py
# A simple pipeline structure that can run different steps in a compute cluster
import azure.core
from azureml.core import Workspace, Datastore, Environment, Experiment, ScriptRunConfig
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline
from azureml.core.runconfig import RunConfiguration
from azureml.widgets import RunDetails
from azureml.data import OutputFileDatasetConfig
import json

ws = Workspace.from_config()

blob_store = Datastore(ws, "workspaceblobstore")    # workspaceblobstore is the default

# Create Experiment
experiment_name = 'train_merge'
exp = Experiment(ws, experiment_name)

# Create Compute Target
# aml_compute_target = 'vdsm-cluster-l80'  # ALL SMALL LETTER, NO underscore, 16ws long only.
aml_compute_target = 'vdsm-cluster-d13'
# try:
#     aml_compute = AmlCompute(ws, aml_compute_target)
#     print("This Compute Target already exist.")
# except ComputeTargetException:
#     print("creating new compute target :", aml_compute_target)
    
#     provisioning_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D13",   
#     # provisioning_config = AmlCompute.provisioning_configuration(vm_size="Standard_L80s_v2", 
#                                                                 min_nodes=0, 
#                                                                 max_nodes=2,
#                                                idle_seconds_before_scaledown=450)    
#     aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
#     aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
# print("Azure Machine Learning Compute Cluster attached now")

# # Or use an existing env which was registered to the ws
# env_name = "vdsm_all" 
# rebuild_env = False   # True if you want to rebuild this env
# if rebuild_env:
#     env = Environment.from_conda_specification(name=env_name, file_path="vdsm_all.yml")
#     env.register(workspace=ws)
# else:
#     envs = Environment.list(workspace=ws)
#     if env_name in envs:
#         # Use an existing environment
#         env = Environment.get(workspace=ws, name=env_name)
#     else:
#         # Create a new env using yml
#         env = Environment.from_conda_specification(name=env_name, file_path="vdsm_all.yml")
#         # Register the environment 
#         env.register(workspace=ws)

env = Environment.get(workspace=ws,name="vdsm_all",version="1")

In [19]:
# A simple pipeline structure that can run different steps in a compute cluster
from azureml.pipeline.steps import PythonScriptStep, ParallelRunStep, ParallelRunConfig
from azureml.pipeline.core.pipeline_output_dataset import PipelineOutputTabularDataset
from azureml.data.dataset_consumption_config import DatasetConsumptionConfig
from azureml.data import OutputFileDatasetConfig
from azureml.pipeline.core.graph import PipelineParameter

pipeline_run_config = ParallelRunConfig(environment=env,
                                        entry_script="train_merge.py",
                                        error_threshold=-1,
                                        output_action="summary_only",
                                        compute_target=aml_compute,
                                        process_count_per_node=1,
                                        # process_count_per_node=20,
                                        # process_count_per_node=5,
                                        run_invocation_timeout=10800,
                                        # node_count=1)
                                        node_count=2)

start_month_pipeline_param = PipelineParameter(name="start_month", default_value="2015-11")
end_month_pipeline_param = PipelineParameter(name="end_month", default_value="2021-07")

ds_list = []
for i in range(1,249,2):
    ds_list.append(DatasetConsumptionConfig(name="row_{}_{}".format(i, i+1), dataset=ws.datasets.get("row_{}_{}".format(i, i+1))))

ds_list.append(DatasetConsumptionConfig(name="row_249", dataset=ws.datasets.get("row_249")))

dataprep_step = ParallelRunStep(
    name="merge_for_training",
    inputs=ds_list,
    arguments=[
        "--start_month", start_month_pipeline_param,
        "--end_month", end_month_pipeline_param,
    ],
    parallel_run_config=pipeline_run_config)

steps = [dataprep_step]

pipeline = Pipeline(workspace=ws, steps=steps)
pipeline_run = exp.submit(pipeline)
print('VDSM training data fusion submitted for execution')
RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion()

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/pipeline/core/_parallel_run_step_base.py:585: UserWarning: 
ParallelRunStep requires azureml-dataset-runtime[fuse,pandas] for tabular dataset.
Please add relevant package in CondaDependencies.
  UserWarning,


Created step merge_for_training [5231eb19][cd18e9b6-5052-4cf5-b7be-36070d256c39], (This step will run and generate new outputs)
Submitted PipelineRun 7127c7e4-d3b9-4325-a2a2-1cbaaef8a191
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/7127c7e4-d3b9-4325-a2a2-1cbaaef8a191?wsid=/subscriptions/7704663e-8944-4300-b951-6462d41ab638/resourcegroups/arg-rt-pacedev-spacetech/workspaces/aml-ws-spacetech001&tid=4341df80-fbe6-41bf-89b0-e6e2379c9c23
VDSM training data fusion submitted for execution


_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

PipelineRunId: 7127c7e4-d3b9-4325-a2a2-1cbaaef8a191
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/7127c7e4-d3b9-4325-a2a2-1cbaaef8a191?wsid=/subscriptions/7704663e-8944-4300-b951-6462d41ab638/resourcegroups/arg-rt-pacedev-spacetech/workspaces/aml-ws-spacetech001&tid=4341df80-fbe6-41bf-89b0-e6e2379c9c23
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: 7d108f0a-69b3-4f70-81b9-b0db51ef73fa
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/7d108f0a-69b3-4f70-81b9-b0db51ef73fa?wsid=/subscriptions/7704663e-8944-4300-b951-6462d41ab638/resourcegroups/arg-rt-pacedev-spacetech/workspaces/aml-ws-spacetech001&tid=4341df80-fbe6-41bf-89b0-e6e2379c9c23
StepRun( merge_for_training ) Status: Running

Streaming azureml-logs/55_azureml-execution-tvmps_70397fd1c2a33553390d31540f35021909acf4f120aedd71ba6186964c6d689b_d.txt
2022-02-03T04:12:50Z Running following command: /bin/bash -c sudo blobfuse /mnt/batch/tasks/shared/LS_root/jobs/aml-ws-spacetech0